**(1) Determine Convex Hull Using Quick Hull Approach**

In [ ]:
from tkinter import Tk, Button, TclError, Button
from random import randint
import tkinter as tk
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import math, os, time, math
from PIL import Image
import shutil

images = []
global iterator
iterator = 0
hull = set()
Output = list()

def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error: {e}")

def create_points(ct,min=0,max=1000):
	return [[randint(min,max),randint(min,max)] \
                for _ in range(ct)]

# Function to handle button clicks and return a string
def on_button_click(button_type):
    if button_type == "Plot_Coordinates":
        return "Plot_Coordinates"
    elif button_type == "Random_Testcase":
        return "Random_Testcase"

# Function to save the result in the variable
def set_label_text(result):
    result_variable.set(result)
    window.destroy()

class CoordinateRecorder:
    def __init__(self, root):
        self.root = root
        self.root.title("Coordinate Recorder")

        self.x_coordinates = []
        self.y_coordinates = []

        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click on the graph to register coordinates')

        self.ax.set_xlim(0, 150)
        self.ax.set_ylim(0, 150)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

        self.canvas.mpl_connect('button_press_event', self.onclick)

        self.submit_button = Button(self.root, text="Submit", command=self.submit_coordinates)
        self.submit_button.pack()

        self.a = None

    def onclick(self, event):
        if event.xdata is not None and event.ydata is not None:
            self.x_coordinates.append(event.xdata)
            self.y_coordinates.append(event.ydata)
            self.ax.scatter(event.xdata, event.ydata, color='red')
            self.canvas.draw()

    def submit_coordinates(self):
        self.a = list(zip(self.x_coordinates, self.y_coordinates))
        self.root.destroy()

    def get_array_a(self):
        return self.a

def findSide(p1, p2, p):
    val = (p[1] - p1[1]) * (p2[0] - p1[0]) - (p2[1] - p1[1]) * (p[0] - p1[0])
    if val > 0:
        return 1
    if val < 0:
        return -1
    return 0

def lineDist(p1, p2, p):
    return abs((p[1] - p1[1]) * (p2[0] - p1[0]) - (p2[1] - p1[1]) * (p[0] - p1[0]))

def quickHull(a, n, p1, p2, side):
    global iterator  # Declare iterator as global
    fig, ax = plt.subplots()
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'og--')
    ind = -1
    max_dist = 0

    
    for i in range(n):
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'og--')
        plt.plot([p1[0], a[i][0]], [p1[1], a[i][1]], 'ob--')
        plt.plot([p2[0], a[i][0]], [p2[1], a[i][1]], 'ob--')

        temp = lineDist(p1, p2, a[i])
        if (findSide(p1, p2, a[i]) == side) and (temp > max_dist):
            ind = i
            max_dist = temp
    ax.set_title('Convex Hull Construction')
    plt.savefig(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))  # Save each plot
    img = Image.open(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))
    images.append(img)
    iterator += 1
    plt.show()

    if ind == -1:
        hull.add("$".join(map(str, p1)))
        hull.add("$".join(map(str, p2)))
        Output.append([p1, p2])
        
        fig, ax = plt.subplots()
        for i in range(1, len(a)):
            plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
        for idx, element in enumerate(Output):
            x1, x2 = element
            plt.plot([x1[0], x2[0]], [x1[1], x2[1]], 'or-')
        ax.set_title('Convex Hull Construction')
        plt.savefig(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))  # Save each plot
        img = Image.open(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))
        images.append(img)
        iterator += 1
        plt.show()

        return

    quickHull(a, n, a[ind], p1, -findSide(a[ind], p1, p2))
    quickHull(a, n, a[ind], p2, -findSide(a[ind], p2, p1))

if __name__ == "__main__":
    # Create the main window
    window = tk.Tk()
    window.title("Input Choices")

    # Set the window size
    window.geometry("400x200")

    # Create a label to display the output
    label = tk.Label(window, text="")
    label.pack(pady=10)

    # Variable to store the result
    result_variable = tk.StringVar()

    # Create two buttons
    mouse_button = tk.Button(window, text="Plot Coordinates on Graph", command=lambda: set_label_text(on_button_click("Plot_Coordinates")))
    mouse_button.pack(pady=5)

    random_button = tk.Button(window, text="Generate Random Testcase", command=lambda: set_label_text(on_button_click("Random_Testcase")))
    random_button.pack(pady=5)
    
    # Start the main event loop
    window.mainloop()

    # Access the result later in the code if needed
    final_result = result_variable.get()

    a = []
    if(final_result == 'Plot_Coordinates'):
        root = Tk()
        app = CoordinateRecorder(root)              # Option (1) : Plotting Coordinates on Graph
        root.mainloop()
        a = app.get_array_a()
    elif(final_result == 'Random_Testcase'):
        a = create_points(200)                      # Option (2) : Define No. of Coordinates
    
    start_time = time.time()

    n = len(a)
    if (n < 3):
        raise Exception("No. of coordinates should be at least 3. Program terminated.")
        
    min_x = 0
    max_x = 0
    for i in range(1, n):
        if a[i][0] < a[min_x][0]:
            min_x = i
        if a[i][0] > a[max_x][0]:
            max_x = i

    xs, ys = zip(*a)
    plt.scatter(xs, ys)
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.show()

    

    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_images_dir = os.path.join(desktop_path, 'CH (Quick Hull)')

    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    empty_folder(output_images_dir)

    quickHull(a, n, a[min_x], a[max_x], 1)
    quickHull(a, n, a[min_x], a[max_x], -1)
    
    gif_file = os.path.join(output_images_dir, 'CH (Quick Hull).gif')
    images[0].save(
        gif_file,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f'GIF saved as {gif_file}')

    print("The points in Convex Hull are:")
    for element in hull:
        x = element.split("$")
        print("(", x[0], ",", x[1], ") ", end=" ")
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\n\nTime Complexity  : O(n log n) = O({n} log {n}) = O({n * math.log10(n)})")
    print(f"Space Complexity : O(n) = O({n})")
    print(f"Execution time   : {elapsed_time} seconds")

**(2) Determine Convex Hull Using Brute Force Approach**

In [ ]:
from random import randint
import tkinter as tk
from tkinter import Tk, Button, TclError
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import math, os, time
from PIL import Image
import shutil

def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error: {e}")

def create_points(ct,min=0,max=1000):
	return [[randint(min,max),randint(min,max)] \
                for _ in range(ct)]

# Function to handle button clicks and return a string
def on_button_click(button_type):
    if button_type == "Plot_Coordinates":
        return "Plot_Coordinates"
    elif button_type == "Random_Testcase":
        return "Random_Testcase"

# Function to save the result in the variable
def set_label_text(result):
    result_variable.set(result)
    window.destroy()

def calculate_angle(x1, y1, x2, y2, x3, y3):
    v1 = [x1 - x2, y1 - y2]
    v2 = [x3 - x2, y3 - y2]
    dot_product = v1[0] * v2[0] + v1[1] * v2[1]
    magnitude_v1 = math.sqrt(v1[0] ** 2 + v1[1] ** 2)
    magnitude_v2 = math.sqrt(v2[0] ** 2 + v2[1] ** 2)
    cos_theta = dot_product / (magnitude_v1 * magnitude_v2)
    theta_rad = math.acos(cos_theta)
    theta_deg = math.degrees(theta_rad)
    return theta_deg

class CoordinateRecorder:
    def __init__(self, root):
        self.root = root
        self.root.title("Coordinate Recorder")

        self.x_coordinates = []
        self.y_coordinates = []

        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click on the graph to register coordinates')

        self.ax.set_xlim(0, 150)
        self.ax.set_ylim(0, 150)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

        self.canvas.mpl_connect('button_press_event', self.onclick)

        self.submit_button = Button(self.root, text="Submit", command=self.submit_coordinates)
        self.submit_button.pack()

        self.a = None

    def onclick(self, event):
        if event.xdata is not None and event.ydata is not None:
            self.x_coordinates.append(event.xdata)
            self.y_coordinates.append(event.ydata)
            self.ax.scatter(event.xdata, event.ydata, color='red')
            self.canvas.draw()

    def submit_coordinates(self):
        self.a = list(zip(self.x_coordinates, self.y_coordinates))
        self.root.destroy()

    def get_array_a(self):
        return self.a

if __name__ == "__main__":
    # Create the main window
    window = tk.Tk()
    window.title("Input Choices")

    # Set the window size
    window.geometry("400x200")

    # Create a label to display the output
    label = tk.Label(window, text="")
    label.pack(pady=10)

    # Variable to store the result
    result_variable = tk.StringVar()

    # Create two buttons
    mouse_button = tk.Button(window, text="Plot Coordinates on Graph", command=lambda: set_label_text(on_button_click("Plot_Coordinates")))
    mouse_button.pack(pady=5)

    random_button = tk.Button(window, text="Generate Random Testcase", command=lambda: set_label_text(on_button_click("Random_Testcase")))
    random_button.pack(pady=5)
    
    # Start the main event loop
    window.mainloop()

    # Access the result later in the code if needed
    final_result = result_variable.get()

    a = []

    if(final_result == 'Plot_Coordinates'):
        root = Tk()
        app = CoordinateRecorder(root)              # Option (1) : Plotting Coordinates on Graph
        root.mainloop()
        a = app.get_array_a()
    elif(final_result == 'Random_Testcase'):
        a = create_points(200)                      # Option (2) : Define No. of Coordinates
 
    start_time = time.time()
    
    Output = []
    visited = [0] * len(a)
    figures = []
    
    n = len(a)
    if (n < 3):
        raise Exception("No. of coordinates should be at least 3. Program terminated.")

    for i in range(len(a) - 1):
        for j in range(i + 1, len(a)):
            x1, y1 = a[i]
            x2, y2 = a[j]
            if y1 > y2:
                temp = a[i]
                a[i] = a[j]
                a[j] = temp
            elif y1 == y2 and x1 > x2:
                temp = a[i]
                a[i] = a[j]
                a[j] = temp

    xs, ys = zip(*a)
    plt.scatter(xs, ys)
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.show()
    end1, end2 = -1, -1
    start1, start2 = a[0]
    previous_x, previous_y = start1 - 1, start2

    Output.append(a[0])
    flag = 1

    plt.xlim(min(point[0] for point in a) - 1, max(point[0] for point in a) + 1)
    plt.ylim(min(point[1] for point in a) - 1, max(point[1] for point in a) + 1)

    while end1 != start1 or end2 != start2:
        for i in range(1, len(a)):
            plt.plot([start1, a[i][0]], [start2, a[i][1]], 'og--')

        if flag == 1:
            end1, end2 = a[0]
        index = -1
        minimum = 361
        distance = -1
        for i in range(len(a)):
            temp1, temp2 = a[i]
            if (start1 != temp1 or start2 != temp2) and visited[i] == 0:
                x1, y1 = previous_x, previous_y
                x2, y2 = start1, start2
                x3, y3 = temp1, temp2
                if ((x1 == x3 and y1 == y3) or (x1 == x2 and y1 == y2) or (x2 == x3 and y2 == y3)) == False:
                    # print(x1, y1, x2, y2, x3, y3)
                    angle = 360 - calculate_angle(x1, y1, x2, y2, x3, y3)
                    if angle < minimum or (angle == minimum and math.sqrt((x2 - x3) ** 2 + (y2 - y3) ** 2) > distance):
                        minimum = angle
                        index = i
                        distance = math.sqrt((x2 - x3) ** 2 + (y2 - y3) ** 2)
        previous_x, previous_y = start1, start2
        start1, start2 = a[index]
        visited[index] = 1
        Output.append(a[index])

        fig, ax = plt.subplots()
        for i in range(1, len(Output)):
            c0 = Output[i - 1]
            c1 = Output[i]
            ax.plot((c0[0], c1[0]), (c0[1], c1[1]), 'or-')
            ax.text(c1[0], c1[1], str(i), ha='right', va='bottom')
        ax.set_title('Convex Hull Construction')
        figures.append(fig)

    plt.show()

    # Save all the figures as individual images on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_images_dir = os.path.join(desktop_path, 'CH (Brute Force)')

    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)
    
    empty_folder(output_images_dir)

    for i, fig in enumerate(figures):
        fig.savefig(os.path.join(output_images_dir, f'figure_{i + 1}.png'))
        plt.close(fig)

    # Create a GIF from the saved images
    images = []
    for i in range(len(figures)):
        img = Image.open(os.path.join(output_images_dir, f'figure_{i + 1}.png'))
        images.append(img)

    gif_file = os.path.join(output_images_dir, 'CH (Brute Force).gif')
    images[0].save(
        gif_file,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )

    print(f'GIF saved as {gif_file}')
    
    Output.pop()
    print("The points in Convex Hull are:")
    print(Output)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\nTime Complexity  : O(n ^ 3) = O({n} ^ 3) = O({n ** 3})")
    print(f"Space Complexity : O(n) = O({n})")
    print(f"Execution time   : {elapsed_time} seconds")

**(3) Determine Convex Hull Using Graham Scan Approach**

In [ ]:
from random import randint
from math import atan2
from time import time
import tkinter as tk
from tkinter import Tk, Button, TclError
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os, shutil, time, math
from PIL import Image

iterator = 0
images = []

def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error: {e}")

def create_points(ct,min=0,max=1000):
	return [[randint(min,max),randint(min,max)] \
                for _ in range(ct)]

def scatter_plot(coords, convex_hull=None):
    global iterator
    fig, ax = plt.subplots()
    for i in range(1, len(coords)):
        plt.plot([coords[0][0], coords[i][0]], [coords[0][1], coords[i][1]], 'og')

    if convex_hull is not None:
        for i in range(1, len(convex_hull)):
            if i == len(convex_hull): i = 0
            c0 = convex_hull[i - 1]
            c1 = convex_hull[i]
            plt.plot((c0[0], c1[0]), (c0[1], c1[1]), 'or-')
    ax.set_title('Convex Hull Construction')
    plt.savefig(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))  # Save each plot
    img = Image.open(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))
    images.append(img)
    iterator += 1
    plt.show()

def polar_angle(p0, p1=None):
	if p1==None: p1=anchor
	y_span=p0[1]-p1[1]
	x_span=p0[0]-p1[0]
	return atan2(y_span,x_span)

def distance(p0 , p1 = None):
	if p1 == None: p1 = anchor
	y_span = p0[1]-p1[1]
	x_span = p0[0]-p1[0]
	return y_span**2 + x_span**2

def det(p1,p2,p3):
	return   (p2[0]-p1[0])*(p3[1]-p1[1]) \
			-(p2[1]-p1[1])*(p3[0]-p1[0])

def quicksort(a):
	if len(a)<=1: return a
	smaller,equal,larger=[],[],[]
	piv_ang=polar_angle(a[randint(0,len(a)-1)])
	for pt in a:
		pt_ang=polar_angle(pt)
		if   pt_ang<piv_ang:  smaller.append(pt)
		elif pt_ang==piv_ang: equal.append(pt)
		else: 				  larger.append(pt)
	return   quicksort(smaller) \
			+sorted(equal,key=distance) \
			+quicksort(larger)

def graham_scan(points,show_progress=False):
	global anchor
	min_idx = None
	for i, (x,y) in enumerate(points):
		if min_idx == None or y < points[min_idx][1]:
			min_idx = i
		if y == points[min_idx][1] and x < points[min_idx][0]:
			min_idx = i

	anchor = points[min_idx]

	sorted_pts = quicksort(points)
	del sorted_pts[sorted_pts.index(anchor)]

	hull = [anchor, sorted_pts[0]] #first two points in hull
	for s in sorted_pts[1:]:
		while det(hull[-2], hull[-1], s) <= 0:	#determinant of stack (top, 2nd top, next sorted point) if <= 0 clockwise turn.
			del hull[-1]
		hull.append(s)
		if show_progress: scatter_plot(points,hull)
		hull.append(anchor)
	return hull

# Function to handle button clicks and return a string
def on_button_click(button_type):
    if button_type == "Plot_Coordinates":
        return "Plot_Coordinates"
    elif button_type == "Random_Testcase":
        return "Random_Testcase"

# Function to save the result in the variable
def set_label_text(result):
    result_variable.set(result)
    window.destroy()

class CoordinateRecorder:
    def __init__(self, root):
        self.root = root
        self.root.title("Coordinate Recorder")

        self.x_coordinates = []
        self.y_coordinates = []

        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click on the graph to register coordinates')

        self.ax.set_xlim(0, 150)
        self.ax.set_ylim(0, 150)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

        self.canvas.mpl_connect('button_press_event', self.onclick)

        self.submit_button = Button(self.root, text="Submit", command=self.submit_coordinates)
        self.submit_button.pack()

        self.a = None

    def onclick(self, event):
        if event.xdata is not None and event.ydata is not None:
            self.x_coordinates.append(event.xdata)
            self.y_coordinates.append(event.ydata)
            self.ax.scatter(event.xdata, event.ydata, color='red')
            self.canvas.draw()

    def submit_coordinates(self):
        self.a = list(zip(self.x_coordinates, self.y_coordinates))
        self.root.destroy()

    def get_array_a(self):
        return self.a

if __name__ == "__main__":
    # Create the main window
    window = tk.Tk()
    window.title("Input Choices")

    # Set the window size
    window.geometry("400x200")

    # Create a label to display the output
    label = tk.Label(window, text="")
    label.pack(pady=10)

    # Variable to store the result
    result_variable = tk.StringVar()

    # Create two buttons
    mouse_button = tk.Button(window, text="Plot Coordinates on Graph", command=lambda: set_label_text(on_button_click("Plot_Coordinates")))
    mouse_button.pack(pady=5)

    random_button = tk.Button(window, text="Generate Random Testcase", command=lambda: set_label_text(on_button_click("Random_Testcase")))
    random_button.pack(pady=5)
    
    # Start the main event loop
    window.mainloop()

    # Access the result later in the code if needed
    final_result = result_variable.get()

    a = []
    if(final_result == 'Plot_Coordinates'):
        root = Tk()
        app = CoordinateRecorder(root)              # Option (1) : Plotting Coordinates on Graph
        root.mainloop()
        a = app.get_array_a()
    elif(final_result == 'Random_Testcase'):
        a = create_points(200)                      # Option (2) : Define No. of Coordinates
    
    start_time = time.time()

    n = len(a)
    if (n < 3):
        raise Exception("No. of coordinates should be at least 3. Program terminated.")

    xs, ys = zip(*a)
    plt.scatter(xs, ys)
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.show()

    # Save all the figures as individual images on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_images_dir = os.path.join(desktop_path, 'CH (Graham Scan)')

    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    empty_folder(output_images_dir)

    hull = graham_scan(a, True)
    scatter_plot(a, hull)

    gif_file = os.path.join(output_images_dir, 'CH (Graham Scan).gif')
    images[0].save(
        gif_file,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )
    print(f'GIF saved as {gif_file}')

    print("The points in Convex Hull are:")
    print(hull)

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\nTime Complexity : O(n log n) = O({n} log {n}) = O({n * math.log10(n)})")
    print(f"Space Complexity : O(n) = O({n})")
    print(f"Execution time: {elapsed_time} seconds")

**(4) Determine Convex Hull Using Jarvis March Approach**

In [ ]:
from random import randint
from math import atan2
from time import time
import tkinter as tk
from tkinter import Tk, Button, TclError
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os, shutil, time
from PIL import Image

iterator = 0
images = []

def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error: {e}")

def create_points(ct, min=0, max=50):
    return [[randint(min, max), randint(min, max)] for _ in range(ct)]

def scatter_plot(coords, convex_hull=None, flag = 0):
    global iterator
    fig, ax = plt.subplots()
    for i in range(1, len(coords)):
        plt.plot([coords[0][0], coords[i][0]], [coords[0][1], coords[i][1]], 'og')

    if convex_hull is not None:
        for i in range(1, len(convex_hull)+1):
            if i == len(convex_hull):
                i = 0

            if i == 1 and flag == 0:
                continue

            c0 = convex_hull[i - 1]
            c1 = convex_hull[i]
            plt.plot((c0[0], c1[0]), (c0[1], c1[1]), 'or-')
    ax.set_title('Convex Hull Construction')
    plt.savefig(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))  # Save each plot
    img = Image.open(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))
    images.append(img)
    iterator += 1
    plt.show()

def orientation(p, q, r):
    val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
    if val == 0:
        return 0  # Collinear
    return 1 if val > 0 else 2  # Clockwise or Counterclockwise

def jarvis_march(points, show_progress=False):
    n = len(points)
    
    hull = []
    l = 0
    for i in range(1, n):
        if points[i][1] < points[l][1] or (points[i][1] == points[l][1] and points[i][0] < points[l][0]):
            l = i

    p = l
    while True:
        hull.append(points[p])
        q = (p + 1) % n
        for i in range(n):
            if orientation(points[p], points[i], points[q]) == 2:
                q = i

        p = q

        if p == l:
            hull.append(points[p])
            break

        if show_progress:
            scatter_plot(points, [points[p], *hull])

    return hull

# Function to handle button clicks and return a string
def on_button_click(button_type):
    if button_type == "Plot_Coordinates":
        return "Plot_Coordinates"
    elif button_type == "Random_Testcase":
        return "Random_Testcase"

# Function to save the result in the variable
def set_label_text(result):
    result_variable.set(result)
    window.destroy()

class CoordinateRecorder:
    def __init__(self, root):
        self.root = root
        self.root.title("Coordinate Recorder")

        self.x_coordinates = []
        self.y_coordinates = []

        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click on the graph to register coordinates')

        self.ax.set_xlim(0, 150)
        self.ax.set_ylim(0, 150)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

        self.canvas.mpl_connect('button_press_event', self.onclick)

        self.submit_button = Button(self.root, text="Submit", command=self.submit_coordinates)
        self.submit_button.pack()

        self.a = None

    def onclick(self, event):
        if event.xdata is not None and event.ydata is not None:
            self.x_coordinates.append(event.xdata)
            self.y_coordinates.append(event.ydata)
            self.ax.scatter(event.xdata, event.ydata, color='red')
            self.canvas.draw()

    def submit_coordinates(self):
        self.a = list(zip(self.x_coordinates, self.y_coordinates))
        self.root.destroy()

    def get_array_a(self):
        return self.a

if __name__ == "__main__":
    # Create the main window
    window = tk.Tk()
    window.title("Input Choices")

    # Set the window size
    window.geometry("400x200")

    # Create a label to display the output
    label = tk.Label(window, text="")
    label.pack(pady=10)

    # Variable to store the result
    result_variable = tk.StringVar()

    # Create two buttons
    mouse_button = tk.Button(window, text="Plot Coordinates on Graph", command=lambda: set_label_text(on_button_click("Plot_Coordinates")))
    mouse_button.pack(pady=5)

    random_button = tk.Button(window, text="Generate Random Testcase", command=lambda: set_label_text(on_button_click("Random_Testcase")))
    random_button.pack(pady=5)
    
    # Start the main event loop
    window.mainloop()

    # Access the result later in the code if needed
    final_result = result_variable.get()

    a = []
    if(final_result == 'Plot_Coordinates'):
        root = Tk()
        app = CoordinateRecorder(root)              # Option (1) : Plotting Coordinates on Graph
        root.mainloop()
        a = app.get_array_a()
    elif(final_result == 'Random_Testcase'):
        a = create_points(200)                      # Option (2) : Define No. of Coordinates
    
    start_time = time.time()

    n = len(a)
    if (n < 3):
        raise Exception("No. of coordinates should be at least 3. Program terminated.")

    xs, ys = zip(*a)
    plt.scatter(xs, ys)
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.show()

    # Save all the figures as individual images on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_images_dir = os.path.join(desktop_path, 'CH (Jarvis March)')

    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    empty_folder(output_images_dir)

    hull = jarvis_march(a, True)
    scatter_plot(a, hull, 1)

    gif_file = os.path.join(output_images_dir, 'CH (Jarvis March).gif')
    images[0].save(
        gif_file,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )
    print(f'GIF saved as {gif_file}')

    print("The points in Convex Hull are:")
    print(hull)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\nTime Complexity  : O(nh) = O({n} * {len(hull)}) = O({n * len(hull)})")
    print(f"Space Complexity : O(n) = O({n})")
    print(f"Execution time   : {elapsed_time} seconds")

**(5) Determine Convex Hull Using Monotone Chain Approach**

In [ ]:
from random import randint
from math import atan2
from time import time
import tkinter as tk
from tkinter import Tk, Button, TclError
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os, shutil, time, math
from PIL import Image

iterator = 0
images = []

def empty_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error: {e}")

def create_points(ct, min=0, max=1000):
    return [[randint(min, max), randint(min, max)] for _ in range(ct)]

def scatter_plot(coords, convex_hull=None):
    global iterator
    fig, ax = plt.subplots()
    for i in range(1, len(coords)):
        plt.plot([coords[0][0], coords[i][0]], [coords[0][1], coords[i][1]], 'og')

    if convex_hull is not None:
        for i in range(1, len(convex_hull)):
            if i == len(convex_hull): i = 0
            c0 = convex_hull[i - 1]
            c1 = convex_hull[i]
            plt.plot((c0[0], c1[0]), (c0[1], c1[1]), 'or-')
    ax.set_title('Convex Hull Construction')
    plt.savefig(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))  # Save each plot
    img = Image.open(os.path.join(output_images_dir, f'plot_{iterator + 1}.png'))
    images.append(img)
    iterator += 1
    plt.show()

def monotone_chain(points, show_progress=False):
    points.sort()  # Sort points lexicographically (by x, then by y)
    
    def build_hull(pts):
        hull = []
        for pt in pts:
            while len(hull) >= 2 and det(hull[-2], hull[-1], pt) <= 0:
                hull.pop()
            hull.append(pt)
            if show_progress:
                scatter_plot(points, hull)
        return hull

    upper_hull = build_hull(points)
    lower_hull = build_hull(points[::-1])
    
    return upper_hull[:-1] + lower_hull[:-1]

def det(p1, p2, p3):
    return (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0])

# Function to handle button clicks and return a string
def on_button_click(button_type):
    if button_type == "Plot_Coordinates":
        return "Plot_Coordinates"
    elif button_type == "Random_Testcase":
        return "Random_Testcase"

# Function to save the result in the variable
def set_label_text(result):
    result_variable.set(result)
    window.destroy()

class CoordinateRecorder:
    def __init__(self, root):
        self.root = root
        self.root.title("Coordinate Recorder")

        self.x_coordinates = []
        self.y_coordinates = []

        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Click on the graph to register coordinates')

        self.ax.set_xlim(0, 150)
        self.ax.set_ylim(0, 150)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

        self.canvas.mpl_connect('button_press_event', self.onclick)

        self.submit_button = Button(self.root, text="Submit", command=self.submit_coordinates)
        self.submit_button.pack()

        self.a = None

    def onclick(self, event):
        if event.xdata is not None and event.ydata is not None:
            self.x_coordinates.append(event.xdata)
            self.y_coordinates.append(event.ydata)
            self.ax.scatter(event.xdata, event.ydata, color='red')
            self.canvas.draw()

    def submit_coordinates(self):
        self.a = list(zip(self.x_coordinates, self.y_coordinates))
        self.root.destroy()

    def get_array_a(self):
        return self.a

if __name__ == "__main__":
    # Create the main window
    window = tk.Tk()
    window.title("Input Choices")

    # Set the window size
    window.geometry("400x200")

    # Create a label to display the output
    label = tk.Label(window, text="")
    label.pack(pady=10)

    # Variable to store the result
    result_variable = tk.StringVar()

    # Create two buttons
    mouse_button = tk.Button(window, text="Plot Coordinates on Graph", command=lambda: set_label_text(on_button_click("Plot_Coordinates")))
    mouse_button.pack(pady=5)

    random_button = tk.Button(window, text="Generate Random Testcase", command=lambda: set_label_text(on_button_click("Random_Testcase")))
    random_button.pack(pady=5)
    
    # Start the main event loop
    window.mainloop()

    # Access the result later in the code if needed
    final_result = result_variable.get()

    a = []
    if(final_result == 'Plot_Coordinates'):
        root = Tk()
        app = CoordinateRecorder(root)              # Option (1) : Plotting Coordinates on Graph
        root.mainloop()
        a = app.get_array_a()
    elif(final_result == 'Random_Testcase'):
        a = create_points(200)                      # Option (2) : Define No. of Coordinates
    
    start_time = time.time()

    n = len(a)
    if (n < 3):
        raise Exception("No. of coordinates should be at least 3. Program terminated.")

    xs, ys = zip(*a)
    plt.scatter(xs, ys)
    for i in range(1, len(a)):
        plt.plot([a[0][0], a[i][0]], [a[0][1], a[i][1]], 'og')
    plt.show()

    # Save all the figures as individual images on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    output_images_dir = os.path.join(desktop_path, 'CH (Monotone Chain)')

    if not os.path.exists(output_images_dir):
        os.makedirs(output_images_dir)

    empty_folder(output_images_dir)
    
    hull = monotone_chain(a, True)
    hull.append(a[0])
    scatter_plot(a, hull)

    gif_file = os.path.join(output_images_dir, 'CH (Monotone Chain).gif')
    images[0].save(
        gif_file,
        save_all=True,
        append_images=images[1:],
        duration=500,
        loop=0
    )
    print(f'GIF saved as {gif_file}')

    print("The points in Convex Hull are:")
    print(hull)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"\nTime Complexity : O(n log n) = O({n} log {n}) = O({n * math.log10(n)})")
    print(f"Space Complexity : O(n) = O({n})")
    print(f"Execution time: {elapsed_time} seconds")